# GEOQUERIES

In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from bson.json_util import dumps

In [20]:
#london_loc = {'type': 'Point', 'coordinates':[ -0.0870022262169723, 51.520624794291884]}
#singapore_loc = {'type': 'Point', 'coordinates':[103.84868192263733, 1.2862863534171092]}
#san_francisco_loc = {'type': 'Point', 'coordinates':[ -122.41049370727525,37.79787988216447]}
london_loc1 = [-0.0870022262169723, 51.520624794291884]
singapore_loc1 = [103.84868192263733, 1.2862863534171092]
san_francisco_loc1 =[ -122.41049370727525,37.79787988216447]

In [21]:
metros = 2500

In [22]:
#Crear la conexión con el database de Londres dentro de MongoDB
client = MongoClient("localhost:27017")
db = client.get_database("Ciudades")

In [23]:
ln = db.get_collection("London")
sg = db.get_collection("Singapore")
sf = db.get_collection("San Francisco")


# FUNCIONES

In [24]:
'''
Esta función recibe la coleccion con la que quieres trabajar, la localizacion establecida, el maximo de distancia
que quieres y la classificación que quieres buscar.
Te devuelve sitios cercanos a la ubicación ajustandose a las condiciones(maxima distancia, por classificacion y por
localización)

'''
def coger_sitios_cercanos(collection, location, Classification, max_distance):
    var_schools = {"$and":[{"location2": {"$near": {"$geometry": location,"$minDistance":0,"$maxDistance": max_distance}}},{"Classification": f'{Classification}'}]}
    encontrar_colegios = collection.find(var_schools)
    return list(encontrar_colegios)
    

In [25]:
#Tipos de classificación/Type
#1. School: Type
#2. Technological Startups
#3. Designing Studios
#4. Leisure Places
#5. Travelling Stations
#6. Parks

In [76]:
# A esta función entra la collection de mongoDB con la que queremos trabajar y la localización establecida.
# Esta función nos crea un dataframe nuevo con la distacia de cada sitio a la localización establecida. 
def get_distances(collection, location1):
    query = [{'$geoNear':{'near':location1,
    'distanceField' : "distance",
    'maxDistance': 1,
    'distanceMultiplier': 6371*1000 ,
    'spherical': True}}]
    geoloc = collection.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    df = pd.DataFrame(response_json)
    df1 = df[['Name', 'Classification', 'Type', 'Country', 'Adress', 'Latitude', 'Longitude', 'location2', 'distance']]
    return df1
    


In [87]:
# En estas 3 variables, tenemos el dataframe con la distacia a nuestro punto inicial en metros.
london_distances = get_distances(ln, london_loc1)
san_francisco_distances = get_distances(sf, san_francisco_loc1)
singapore_distances = get_distances(sg, singapore_loc1)

In [89]:
london_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Finsbury Square,Parks,Park,GB,Finsbury Sq (Finsbury Pavement),51.520694,-0.086512,"{'type': 'Point', 'coordinates': [-0.086511969...",34.776274
1,Finch's,Leisure Places,Pub,GB,12A Finsbury Sq (City Rd),51.521709,-0.087048,"{'type': 'Point', 'coordinates': [-0.087048251...",120.590198


In [90]:
san_francisco_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Jean Parker Elementary,School,Elementary School,US,840 Broadway,37.797641,-122.411043,"{'type': 'Point', 'coordinates': [-122.411043,...",55.089767
1,Broadway Tunnel East Mini Park,Parks,Park,US,967 Broadway (at Himmelman),37.797354,-122.410502,"{'type': 'Point', 'coordinates': [-122.4105024...",58.444345


In [91]:
singapore_distances.head(2)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location2,distance
0,Liquid 40,Leisure Places,Pub,SG,Circular Rd,1.286381,103.848842,"{'type': 'Point', 'coordinates': [103.84884158...",20.615074
1,Audition pub,Leisure Places,Pub,SG,76 Circular Road,1.286644,103.848966,"{'type': 'Point', 'coordinates': [103.84896614...",50.772528


# PONDERACION

In [ ]:
# Para escoger la localización, debemos dar importancia a las condiciones escogidas

In [92]:
parks_pond = 0.05
school_pond = 0.30
technological_Startups_pond = 0.05
designing_Studios_pond = 0.25
travelling_Stations_pond = 0.2
leisure_pond = 0.15